In [75]:

%pprint

Pretty printing has been turned OFF


['Hospital_Beds_Per_Thousand_2015', 'Hospital_Beds_Per_Thousand_2014', 'Hospital_Beds_Per_Thousand_2016', 'Country_Name', 'Hospital_Beds_Per_Thousand_2013', 'Hospital_Beds_Per_Thousand_2017']


---
# Scrape the Data

In [1]:

%run ../../load_magic/storage.py
%run ../../load_magic/dataframes.py
s = Storage()


----
## Create US State Hospital Capacity DataFrame

In [3]:

try:
    states_hospital_capacity_df = s.load_object('states_hospital_capacity_df')
except:
    tables_url = 'https://www.ahd.com/state_statistics.html'
    tables_list = get_page_tables(tables_url, verbose=False)
    states_hospital_capacity_df = tables_list[1].copy()
    if states_hospital_capacity_df.shape[0] < 50:
        tables_url = os.path.join(s.data_folder, 'html', 'states_hospital_capacity.html')
        tables_list = get_page_tables(tables_url, verbose=True)
        states_hospital_capacity_df = tables_list[0].copy()
    states_hospital_capacity_df = states_hospital_capacity_df.iloc[:-1]
    states_hospital_capacity_df.columns = ['State_Name', 'Hospital_Count', 'Staffed_Beds_Count', 'Discharges_Count',
                                           'Patient_Days', 'Gross_Patient_Revenue_Thousands']
    for cn in states_hospital_capacity_df.columns:
        if cn == 'State_Name':
            states_hospital_capacity_df[cn] = states_hospital_capacity_df[cn].map(lambda x: str(x).split('-')[1].strip())
        else:
            states_hospital_capacity_df[cn] = states_hospital_capacity_df[cn].map(lambda x: re.sub('[^\d\.]+', '', str(x)))
            states_hospital_capacity_df[cn] = pd.to_numeric(states_hospital_capacity_df[cn], errors='coerce')
    states_hospital_capacity_df.set_index('State_Name', drop=True, inplace=True)
    s.store_objects(states_hospital_capacity_df=states_hospital_capacity_df)
states_hospital_capacity_df

No pickle exists at D:\Documents\Repositories\notebooks\covid19\saves\pickle\states_hospital_capacity_df.pickle - attempting to load as csv.
No csv exists at D:\Documents\Repositories\notebooks\covid19\saves\csv\states_hospital_capacity_df.csv - attempting to download from URL.
Pickling to D:\Documents\Repositories\notebooks\covid19\saves\pickle\states_hospital_capacity_df.pickle


,Hospital_Count,Staffed_Beds_Count,Discharges_Count,Patient_Days,Gross_Patient_Revenue_Thousands
State_Name,,,,,
Alaska,10,1235,45209,243272,5374320
Alabama,89,15330,592368,2818440,60726101
Arkansas,52,7873,315200,1385087,27777922
American Samoa,1,0,0,0,0
Arizona,73,13416,610133,2660941,79447094
California,343,74624,3084295,14352052,484959610
Colorado,55,8189,394733,1755507,67533414
Connecticut,34,8798,368120,1742965,40204715
Washington D.C.,7,2117,93225,586450,12585158



----
## Create Global Hospital Capacity DataFrame

In [78]:

# https://data.worldbank.org/indicator/SH.MED.BEDS.ZS
try:
    world_hospital_capacity_df = s.load_object('world_hospital_capacity_df')
    columns_list = world_hospital_capacity_df.columns.tolist()
    country_list = world_hospital_capacity_df.Country_Name.unique().tolist()
    mask_series = world_hospital_capacity_df.Country_Name.isin(country_list)
except:
    file_path = os.path.join(s.data_folder, 'xlsx', 'API_SH.MED.BEDS.ZS_DS2_en_excel_v2_988704.xlsx')
    worldbank_df = pd.read_excel(file_path)
    worldbank_df.columns = [str(cn).replace(' ', '_').replace('.0', '') for cn in worldbank_df.iloc[2]]
    worldbank_df = worldbank_df.iloc[3:]
    columns_list = ['Country_Name']
    columns_dict = {}
    for i in range(1960, 2021):
        if str(i) in worldbank_df.columns:
            cn = f'Hospital_Beds_Per_Thousand_{i}'
            columns_list.append(cn)
            columns_dict[str(i)] = cn
    worldbank_df = worldbank_df.rename(columns=columns_dict)
    worldbank_name = 'Hong Kong SAR, China'
    wikipedia_name = 'Hong Kong'
    mask_series = (worldbank_df.Country_Name == worldbank_name)
    worldbank_df.loc[mask_series, 'Country_Name'] = wikipedia_name
    worldbank_name = 'Russian Federation'
    wikipedia_name = 'Russia'
    mask_series = (worldbank_df.Country_Name == worldbank_name)
    worldbank_df.loc[mask_series, 'Country_Name'] = wikipedia_name
    worldbank_name = 'Slovak Republic'
    wikipedia_name = 'Slovakia'
    mask_series = (worldbank_df.Country_Name == worldbank_name)
    worldbank_df.loc[mask_series, 'Country_Name'] = wikipedia_name
    worldbank_name = 'Korea, Rep.'
    wikipedia_name = 'South Korea'
    mask_series = (worldbank_df.Country_Name == worldbank_name)
    worldbank_df.loc[mask_series, 'Country_Name'] = wikipedia_name
    tables_url = 'https://en.wikipedia.org/wiki/List_of_countries_by_hospital_beds'
    tables_list = get_page_tables(tables_url, verbose=False)
    wikipedia_df = tables_list[0].copy()
    wikipedia_df.columns = ['Rank', 'Country_Name', 'Continent_Name', 'Hospital_Beds_Per_Thousand_2013',
                            'Hospital_Beds_Per_Thousand_2014', 'Hospital_Beds_Per_Thousand_2015',
                            'Hospital_Beds_Per_Thousand_2016', 'Hospital_Beds_Per_Thousand_2017',
                            'Change_From_Previous_Year_Average_Actual',
                            'Change_From_Previous_Year_Average_Proportional_Percent', 'Occupancy_Percent',
                            'ICU_Beds_Per_Thousand', 'Ventilators']
    for cn in wikipedia_df.columns:
        if cn in ['Country_Name', 'Continent_Name']:
            wikipedia_df[cn] = wikipedia_df[cn].map(lambda x: str(x).split('(')[0].strip())
        else:
            f = lambda x: re.sub('[^\d\.−]+', '', str(x).split('(')[0].split('[')[0]).replace('−', '-')
            wikipedia_df[cn] = wikipedia_df[cn].map(f)
            wikipedia_df[cn] = pd.to_numeric(wikipedia_df[cn], errors='coerce')
    columns_list = list(set(df.columns).intersection(set(wikipedia_df.columns)))
    for cn in columns_list:
        for idx in wikipedia_df.index:
            cv = wikipedia_df.loc[idx, cn]
            country_name = wikipedia_df.loc[idx, 'Country_Name']
            mask_series = (worldbank_df.Country_Name == country_name)
            ocv = worldbank_df.loc[mask_series, cn].squeeze()
            if str(cv) != 'nan':
                if str(ocv) == 'nan':
                    worldbank_df.loc[mask_series, cn] = cv
                else:
                    wikipedia_df.loc[idx, cn] = ocv
            else:
                if str(ocv) != 'nan':
                    wikipedia_df.loc[idx, cn] = ocv
    country_list = wikipedia_df.Country_Name.unique().tolist()
    world_hospital_capacity_df = wikipedia_df.merge(worldbank_df, how='outer', on=columns_list)
    world_hospital_capacity_df.set_index('Country_Name', drop=True, inplace=True)
    s.store_objects(world_hospital_capacity_df=world_hospital_capacity_df)
    columns_list = ['Hospital_Beds_Per_Thousand_2013', 'Hospital_Beds_Per_Thousand_2015', 'Hospital_Beds_Per_Thousand_2014',
                    'Hospital_Beds_Per_Thousand_2016', 'Hospital_Beds_Per_Thousand_2017']
    mask_series = world_hospital_capacity_df.index.isin(country_list)
world_hospital_capacity_df[mask_series][columns_list].sample(5).T

Pickling to D:\Documents\Repositories\notebooks\covid19\saves\pickle\world_hospital_capacity_df.pickle


Country_Name,United States,Czech Republic,Canada,Poland,United Kingdom
Hospital_Beds_Per_Thousand_2013,2.90,6.50,2.71,6.50,2.80
Hospital_Beds_Per_Thousand_2015,2.80,6.50,2.61,6.63,2.61
Hospital_Beds_Per_Thousand_2014,2.83,6.68,2.67,6.63,2.73
Hospital_Beds_Per_Thousand_2016,2.77,6.66,2.58,6.64,2.57
Hospital_Beds_Per_Thousand_2017,NaN,6.63,2.52,6.62,2.54
